In [1]:
import pandas as pd
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName('pyspark') \
    .getOrCreate()
# 原始数据 
test = spark.createDataFrame([('001','1',100,87,67,83,98), ('002','2',87,81,90,83,83), ('003','3',86,91,83,89,63),
                            ('004','2',65,87,94,73,88), ('005','1',76,62,89,81,98), ('006','3',84,82,85,73,99),
                            ('007','3',56,76,63,72,87), ('008','1',55,62,46,78,71), ('009','2',63,72,87,98,64)],                          
                             ['number','class','language','math','english','physic','chemical'])
test.show()

+------+-----+--------+----+-------+------+--------+
|number|class|language|math|english|physic|chemical|
+------+-----+--------+----+-------+------+--------+
|   001|    1|     100|  87|     67|    83|      98|
|   002|    2|      87|  81|     90|    83|      83|
|   003|    3|      86|  91|     83|    89|      63|
|   004|    2|      65|  87|     94|    73|      88|
|   005|    1|      76|  62|     89|    81|      98|
|   006|    3|      84|  82|     85|    73|      99|
|   007|    3|      56|  76|     63|    72|      87|
|   008|    1|      55|  62|     46|    78|      71|
|   009|    2|      63|  72|     87|    98|      64|
+------+-----+--------+----+-------+------+--------+



In [2]:
# 查看列的类型 ，同pandas
test.dtypes

[('number', 'string'),
 ('class', 'string'),
 ('language', 'bigint'),
 ('math', 'bigint'),
 ('english', 'bigint'),
 ('physic', 'bigint'),
 ('chemical', 'bigint')]

In [3]:
# 查看有哪些列 ，同pandas
test.columns

['number', 'class', 'language', 'math', 'english', 'physic', 'chemical']

In [4]:
# 行数
test.count()

9

In [6]:
# 选择一列的几种方式，比较麻烦，不像pandas直接用df['cols']就可以了
# 需要在filter,select等操作符中才能使用
test.select('language').show()
test.select(test.language).show()
test.select(test[2]).show()
test.select(test['language']).show()
test.select(test['language']>=90).show()

+--------+
|language|
+--------+
|     100|
|      87|
|      86|
|      65|
|      76|
|      84|
|      56|
|      55|
|      63|
+--------+

+--------+
|language|
+--------+
|     100|
|      87|
|      86|
|      65|
|      76|
|      84|
|      56|
|      55|
|      63|
+--------+

+--------+
|language|
+--------+
|     100|
|      87|
|      86|
|      65|
|      76|
|      84|
|      56|
|      55|
|      63|
+--------+

+--------+
|language|
+--------+
|     100|
|      87|
|      86|
|      65|
|      76|
|      84|
|      56|
|      55|
|      63|
+--------+

+----------------+
|(language >= 90)|
+----------------+
|            true|
|           false|
|           false|
|           false|
|           false|
|           false|
|           false|
|           false|
|           false|
+----------------+



In [7]:
# 选择几列的方法
test.select('class','language').show()

+-----+--------+
|class|language|
+-----+--------+
|    1|     100|
|    2|      87|
|    3|      86|
|    2|      65|
|    1|      76|
|    3|      84|
|    3|      56|
|    1|      55|
|    2|      63|
+-----+--------+



In [8]:
# 多列选择和切片
test.select('class','language') \
        .select(test['language']>80).show()

+---------------+
|(language > 80)|
+---------------+
|           true|
|           true|
|           true|
|          false|
|          false|
|           true|
|          false|
|          false|
|          false|
+---------------+



In [9]:
# between 范围选择
test.filter(test.language.between(60,70) )\
        .select(test.language.alias('mid_grade')).show()

+---------+
|mid_grade|
+---------+
|       65|
|       63|
+---------+



In [10]:
# filter运行类SQL
test.filter("class=1").show()
test.filter("class like '1%'").show()

+------+-----+--------+----+-------+------+--------+
|number|class|language|math|english|physic|chemical|
+------+-----+--------+----+-------+------+--------+
|   001|    1|     100|  87|     67|    83|      98|
|   005|    1|      76|  62|     89|    81|      98|
|   008|    1|      55|  62|     46|    78|      71|
+------+-----+--------+----+-------+------+--------+

+------+-----+--------+----+-------+------+--------+
|number|class|language|math|english|physic|chemical|
+------+-----+--------+----+-------+------+--------+
|   001|    1|     100|  87|     67|    83|      98|
|   005|    1|      76|  62|     89|    81|      98|
|   008|    1|      55|  62|     46|    78|      71|
+------+-----+--------+----+-------+------+--------+



In [11]:
# 直接使用SQL语法
# 首先dataframe注册为临时表，然后执行SQL查询
test.createOrReplaceTempView("test_df")
spark.sql("select count(*) number_count from test_df").show()

+------------+
|number_count|
+------------+
|           9|
+------------+



In [22]:
# spark排序
test.sort('language',ascending=False).show()

+------+-----+--------+----+-------+------+--------+
|number|class|language|math|english|physic|chemical|
+------+-----+--------+----+-------+------+--------+
|   001|    1|     100|  87|     67|    83|      98|
|   002|    2|      87|  81|     90|    83|      83|
|   003|    3|      86|  91|     83|    89|      63|
|   006|    3|      84|  82|     85|    73|      99|
|   005|    1|      76|  62|     89|    81|      98|
|   004|    2|      65|  87|     94|    73|      88|
|   009|    2|      63|  72|     87|    98|      64|
|   007|    3|      56|  76|     63|    72|      87|
|   008|    1|      55|  62|     46|    78|      71|
+------+-----+--------+----+-------+------+--------+



In [23]:
# 1.生成测试数据
import numpy as np
import pandas as pd

df=pd.DataFrame(np.random.rand(5,5),columns=['a','b','c','d','e'])\
    .applymap(lambda x: int(x*10))
df.iloc[2,2]=np.nan

spark_df = spark.createDataFrame(df)
spark_df.show()

# 2.删除有缺失值的行
df2 = spark_df.dropna()
df2.show()

# 3.或者
spark_df=spark_df.na.drop()


+---+---+---+---+---+
|  a|  b|  c|  d|  e|
+---+---+---+---+---+
|  5|  3|1.0|  9|  8|
|  8|  1|5.0|  2|  5|
|  5|  4|NaN|  8|  8|
|  9|  1|6.0|  8|  2|
|  6|  4|7.0|  3|  3|
+---+---+---+---+---+

+---+---+---+---+---+
|  a|  b|  c|  d|  e|
+---+---+---+---+---+
|  5|  3|1.0|  9|  8|
|  8|  1|5.0|  2|  5|
|  9|  1|6.0|  8|  2|
|  6|  4|7.0|  3|  3|
+---+---+---+---+---+



In [25]:
#分组统计1
test.groupBy("class").count().show()
# 分组计算2：应用多函数
import pyspark.sql.functions as func
test.groupBy("class").agg(func.max("language"), func.avg("language")).show()

+-----+-----+
|class|count|
+-----+-----+
|    3|    3|
|    1|    3|
|    2|    3|
+-----+-----+

+-----+-------------+-----------------+
|class|max(language)|    avg(language)|
+-----+-------------+-----------------+
|    3|           86|75.33333333333333|
|    1|          100|             77.0|
|    2|           87|71.66666666666667|
+-----+-------------+-----------------+



In [27]:
# 1.生成测试数据
employees = [(1, "John", 25), (2, "Ray", 35), (3,"Mike", 24), (4, "Jane", 28), 
             (5, "Kevin", 26), 
             (6, "Vincent", 35), (7,"James", 38), (8, "Shane", 32), 
             (9, "Larry", 29), (10, "Kimberly", 29),
             (11, "Alex", 28), (12, "Garry", 25), (13, "Max",31)]
employees=spark.createDataFrame(employees, schema=["emp_id","name","age"])
employees.show()

salary=[(1,1000),(2,2000),(3,3000),(4,4000)]
salary=spark.createDataFrame(salary, schema=["emp_id","salary"])
salary.show()

department=[(1,1000),(2,2000),(3,3000),(4,4000)]
department=spark.createDataFrame(department, schema=["emp_id","departement"])
department.show()

# 2.连接
# join默认是内连接，最终结果会存在重复列名
# 如果是pandas,重复列会用_x,_y等后缀标识出来，但spark不会
# join会在最后的dataframe中存在重复列

final_data = employees.join(salary, employees.emp_id == salary.emp_id, 
                            how='left')\
    .join(department, employees.emp_id==department.emp_id)
final_data.show()

# 3.如果两边的关联字段名相同，也可以省去很多麻烦
final_data = employees.join(salary, on='emp_id', how='left')\
    .join(department, on='emp_id', how='left')
final_data.show()

+------+--------+---+
|emp_id|    name|age|
+------+--------+---+
|     1|    John| 25|
|     2|     Ray| 35|
|     3|    Mike| 24|
|     4|    Jane| 28|
|     5|   Kevin| 26|
|     6| Vincent| 35|
|     7|   James| 38|
|     8|   Shane| 32|
|     9|   Larry| 29|
|    10|Kimberly| 29|
|    11|    Alex| 28|
|    12|   Garry| 25|
|    13|     Max| 31|
+------+--------+---+

+------+------+
|emp_id|salary|
+------+------+
|     1|  1000|
|     2|  2000|
|     3|  3000|
|     4|  4000|
+------+------+

+------+-----------+
|emp_id|departement|
+------+-----------+
|     1|       1000|
|     2|       2000|
|     3|       3000|
|     4|       4000|
+------+-----------+

+------+----+---+------+------+------+-----------+
|emp_id|name|age|emp_id|salary|emp_id|departement|
+------+----+---+------+------+------+-----------+
|     1|John| 25|     1|  1000|     1|       1000|
|     3|Mike| 24|     3|  3000|     3|       3000|
|     2| Ray| 35|     2|  2000|     2|       2000|
|     4|Jane| 28|    

In [29]:
# 用均值替换缺失值
from pyspark.sql import functions as func  # 导入spark内置函数
# 计算缺失值，collect()函数将数据返回到driver端，为Row对象，[0]可以获取Row的值
mean_salary = final_data.select(func.mean('salary')).collect()[0][0]
clean_data = final_data.na.fill({'salary':mean_salary})
clean_data.show()

+------+--------+---+------+-----------+
|emp_id|    name|age|salary|departement|
+------+--------+---+------+-----------+
|     7|   James| 38|  2500|       null|
|     6| Vincent| 35|  2500|       null|
|     9|   Larry| 29|  2500|       null|
|     5|   Kevin| 26|  2500|       null|
|     1|    John| 25|  1000|       1000|
|    10|Kimberly| 29|  2500|       null|
|     3|    Mike| 24|  3000|       3000|
|    12|   Garry| 25|  2500|       null|
|     8|   Shane| 32|  2500|       null|
|    11|    Alex| 28|  2500|       null|
|     2|     Ray| 35|  2000|       2000|
|     4|    Jane| 28|  4000|       4000|
|    13|     Max| 31|  2500|       null|
+------+--------+---+------+-----------+

